In [35]:
import torch
from pathlib import Path
from torch import nn
import pickle
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(2704, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 15)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Assuming you know the configuration of the best model
best_model_config = {
    'l1': 256,  # example value, use the actual value from best_trial.config
    'l2': 512   # example value, use the actual value from best_trial.config
}

# Initialize the model
loaded_model = Net(best_model_config['l1'], best_model_config['l2'])

# Load the model state
model_filename = 'best_model.pkl'
with open(model_filename, 'rb') as f:
    loaded_model = pickle.load(f)

print("Model loaded successfully")

Model loaded successfully


In [45]:
import pickle
import torch
from torch import nn
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Function to preprocess images
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    return image.unsqueeze(0)  # Add batch dimension

# Preprocess your image
image_tensor = preprocess_image('input_1_2_5.jpg')

# Make predictions
loaded_model.eval()  # Set the model to evaluation mode
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model.to(device)
image_tensor = image_tensor.to(device)

with torch.no_grad():
    outputs = loaded_model(image_tensor)
    _, predicted = torch.max(outputs.data, 1)

print('Predicted class:', predicted.item())

Predicted class: 6
